In [2]:
# load modules/packages
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sb

from IPython.display import Image

import warnings
warnings.simplefilter('ignore', category=RuntimeWarning)

# local util modules
import plotters

In [3]:
## load data, prepare data for pipeline ## 

# to load my custom vdW heterostructure dataset (uses custom package 'hetml' - not fully released/published)
from hetml.data.dataloaders import load_featureset, load_prediction_dataset, quickload_datatables

# load the features/targets for predicting the ionization energy (IE)
TARGET = 'IE'
X0 ,_ , Xall   = load_featureset(target=TARGET)
targets = load_prediction_dataset(target=TARGET)
master, all_targets = quickload_datatables() 

# get lasso coefs (from previous work)
fpath = f'../data/lasso_coefs/lasso_coefs_{TARGET}.csv'
coefs = pd.read_csv(fpath, index_col=0)

# prepare data
X = X0.copy() 
X = pd.concat((X, Xall))
y = pd.Series(data=targets.y_pred, index=targets.index)
X = X.reindex(y.index)

# drop type-III materials and large latmm materials
X = X.reindex(master[master.band_alignment.isin(['I','II'])].index) 
X = X.reindex(master[master.latmm < 3.0].index)
y = y.reindex(X.index)
X = X.dropna(); y = y.dropna()
y = y.reindex(X.index).dropna() 

# use only lasso selected features
X_all_feats = X.copy() 
selected_feats = coefs.abs().iloc[:10].index.tolist()  # top 10 features only
selected_feats = coefs.abs().sort_values('lasso_coefs', ascending=False)[:10].index  # top 10 features only
X = X.reindex(columns=selected_feats)


# final features/targets
features = X.copy() 
targets  = y.copy() 
print(features.shape, targets.shape)

returning --> master | targets

Preprocessing Steps:
Feature Space Dim: (6332, 342)
Targets Dim: (790, 37)
	-building Anderson's Rule classes
	-building stacking configuration classes
	-using AUB bilayers, transforming to binary classes
	-dropping feature columns: (6332, 90)
	-dropping metal bilayers: (689, 37)
	-dropping Type III bilayers: (600, 37)
II    310
I     290
Name: band_alignment, dtype: int64
	-dropping bilayers w/ ILD < 2.5: (595, 37)
	-dropping bilayers w/ Eb > 40 meV: (595, 37)
	-dropping bilayers w/ charge transf. > 1 |e|: (595, 37)
	-one-hot-encod band alignment: (6332, 91)

Final --> Feature Space Dim: (6332, 91)
Final --> Targets Dim: (595, 37)

target = 'IE'  stacktype = 'AUB'
x dim: (595, 91)
y dim: (595,)
unlabeled-X dim: (5737, 91)


All Features (Pre-Feature Selection) (p=91):
['avg_gap_nosoc', 'avg_evac', 'avg_hform', 'avg_emass1', 'avg_emass2', 'avg_efermi_hse_nosoc', 'avg_cbm', 'avg_vbm', 'avg_excitonmass1', 'avg_excitonmass2', 'avg_alphax', 'avg_alphaz', 'av

In [4]:
import pandas as pd

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV, LinearRegression, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [5]:
# preprocessing pipeline
NUM_FEATS = X_all_feats.columns.tolist() 

# LASSO feature selection model
lasso_fs = SelectFromModel(estimator=
        LassoCV(
            n_alphas=1000,
            max_iter=4500,
            n_jobs=-1,
            cv=5, 
            normalize=False) #data is already normalized/scaled
) 

# stacked regressor
basemodels = [
    ('GBR',    GradientBoostingRegressor()) ,
    ('Ridge',  RidgeCV() ) ,
    ('SVR',    SVR(kernel='linear')) ,
    ('KRR',    KernelRidge(kernel='poly'))
    ]
sem = StackingRegressor(
        estimators=basemodels, 
        final_estimator=LinearRegression())

## Pipelines and transformers

num_pipe = Pipeline(steps=[
    ('scale_features', StandardScaler() )
])

preprocessing_pipe = ColumnTransformer(transformers=[
    ('num_features', num_pipe, NUM_FEATS )
], remainder='drop')


lasso_featsel_pipe = Pipeline(steps=[
    ('preprocessing',   preprocessing_pipe),
    ('lasso_feat_sel',  lasso_fs),
])

## full pipelines 
full_pipeline_sem = Pipeline(steps=[
    ('preprocessing',       preprocessing_pipe),
    ('lasso_feature_sel',   lasso_fs),
    ('model',               sem)
])


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate

xtrain,xtest, ytrain,ytest = train_test_split(X_all_feats, y, train_size=0.85)

scoring = 'neg_mean_absolute_error'
cv_score_sem = cross_validate(full_pipeline_sem, xtrain, ytrain, cv=5, return_estimator=True, scoring=scoring)
cv_score_sem


{'fit_time': array([31.80210114, 30.69763708, 31.33507633, 32.62369537, 32.46002364]),
 'score_time': array([0.04590297, 0.04113364, 0.04174328, 0.040869  , 0.04221702]),
 'estimator': [Pipeline(steps=[('preprocessing',
                   ColumnTransformer(transformers=[('num_features',
                                                    Pipeline(steps=[('scale_features',
                                                                     StandardScaler())]),
                                                    ['avg_gap_nosoc', 'avg_evac',
                                                     'avg_hform', 'avg_emass1',
                                                     'avg_emass2',
                                                     'avg_efermi_hse_nosoc',
                                                     'avg_cbm', 'avg_vbm',
                                                     'avg_excitonmass1',
                                                     'avg_excitonmass2',
        

In [6]:
best_sem = cv_score_sem.pop('estimator')[ cv_score_sem['test_score'].argmin() ]
best_sem 

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_features',
                                                  Pipeline(steps=[('scale_features',
                                                                   StandardScaler())]),
                                                  ['avg_gap_nosoc', 'avg_evac',
                                                   'avg_hform', 'avg_emass1',
                                                   'avg_emass2',
                                                   'avg_efermi_hse_nosoc',
                                                   'avg_cbm', 'avg_vbm',
                                                   'avg_excitonmass1',
                                                   'avg_excitonmass2',
                                                   'avg_alphax', 'avg_alphaz',
                                                   'avg_bse_binding',
                                                   'avg_c_11', 'a

In [15]:
from sklearn.model_selection import GridSearchCV
# model/algo
gpr  = GaussianProcessRegressor(alpha=1e-10) # using default kernel (RBF)

# preprocessing pipeline
num_features = X.columns.tolist() 
num_pipe = Pipeline(steps=[
    ('scale_features', StandardScaler() )
])

preprocessing_pipe = ColumnTransformer(transformers=[
    ('preprocessing_num', num_pipe, num_features)
])

# full pipeline

full_pipeline_gpr = Pipeline(steps=[
    ('preprocessing',   preprocessing_pipe),
    ('model',           gpr)
])


scoring = 'neg_mean_absolute_error'
cv_score_gpr = cross_validate(full_pipeline_gpr, xtrain, ytrain, cv=5, return_estimator=True, scoring=scoring)
print(cv_score_gpr)
print(cv_score_gpr['test_score'].mean() *-1)

{'fit_time': array([0.49520087, 0.34897852, 0.38471723, 0.37289453, 0.37361383]), 'score_time': array([0.02680945, 0.01849651, 0.01789308, 0.01767564, 0.02176571]), 'estimator': [Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('preprocessing_num',
                                                  Pipeline(steps=[('scale_features',
                                                                   StandardScaler())]),
                                                  ['max_vbm',
                                                   'mean_vbmsite_atomic_vol',
                                                   'avg_c_12', 'avg_cbm_d',
                                                   'avg_vbm_char_p',
                                                   'avg_lattice_param',
                                                   'avg_emass2',
                                                   'PymatgenData maximum '
                                              

In [14]:
"""
how to otpimie hyperparameters using a nest cross-validation. 
The default kernel and alpha seem to work fine so will bypass this in interest of time
"""
# # hyperparameter tuning
# from sklearn.gaussian_process import kernels
# param_grid = {
#     'model__alpha': [1e-1, 1e-5, 1e-10],
#     'model__kernel': [kernels.DotProduct(), kernels.ExpSineSquared(), kernels.Matern() ]
#     }

# xtrain,xtest, ytrain,ytest = train_test_split(X, y, train_size=0.85)

# grid_search  = GridSearchCV(full_pipeline_gpr,param_grid, cv=5)


# scoring = 'neg_mean_absolute_error'
# cv_score_gpr = cross_validate(grid_search, xtrain, ytrain, cv=5, return_estimator=True, scoring=scoring)
# print(cv_score_gpr)
# print(cv_score_gpr['test_score'].mean() *-1)

'\nhow to otpimie hyperparameters using a nest cross-validation. \nThe default kernel and alpha seem to work fine so will bypass this in interest of time\n'

In [9]:
best_gpr = cv_score_gpr.pop('estimator')[cv_score_gpr['test_score'].argmin() ]
best_gpr

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('preprocessing_num',
                                                                         Pipeline(steps=[('scale_features',
                                                                                          StandardScaler())]),
                                                                         ['max_vbm',
                                                                          'mean_vbmsite_atomic_vol',
                                                                          'avg_c_12',
                                                                          'avg_cbm_d',
                                                                          'avg_vbm_char_p',
                                                                          'avg_lattice_param',
                                                                 

In [10]:
from lolopy.learners import RandomForestRegressor

# model/algo
rf = RandomForestRegressor() 

# preprocessing pipeline
num_features = X.columns.tolist() 
num_pipe = Pipeline(steps=[
    ('scale_features', StandardScaler() )
])

preprocessing_pipe = ColumnTransformer(transformers=[
    ('preprocessing_num', num_pipe, num_features)
])

# full pipeline
full_pipeline_rf = Pipeline(steps=[
    ('model',           rf)
])

xtrain,xtest, ytrain,ytest = train_test_split(X, y, train_size=0.85)

scoring = 'neg_mean_absolute_error'
cv_score_rf = cross_validate(full_pipeline_rf, xtrain.to_numpy(), ytrain.to_numpy(), cv=5, return_estimator=True, scoring=scoring )
cv_score_rf

{'fit_time': array([35.41870165, 30.73100781, 31.51309609, 34.6600163 , 32.91029167]),
 'score_time': array([0.64313555, 0.42985058, 0.42447734, 0.76751876, 0.45012569]),
 'estimator': [Pipeline(steps=[('model', RandomForestRegressor())]),
  Pipeline(steps=[('model', RandomForestRegressor())]),
  Pipeline(steps=[('model', RandomForestRegressor())]),
  Pipeline(steps=[('model', RandomForestRegressor())]),
  Pipeline(steps=[('model', RandomForestRegressor())])],
 'test_score': array([-0.03213236, -0.03114404, -0.02935451, -0.03380064, -0.0357899 ])}

In [11]:
best_rf = cv_score_rf.pop('estimator')[ cv_score_rf['test_score'].argmin() ]
best_rf.score(xtest, ytest)

0.9943922730590421

In [12]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(ytest, best_rf.predict(xtest))

0.03446130457969282